In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [ ]:
import optuna
optuna.logging.disable_default_handler()

In [3]:
from fastai.vision import *

In [4]:
bs = 32
transform = transforms.Compose(
[transforms.ToTensor(),
transforms.Normalize((.5,),(.5,))
]
)

In [5]:
train_ds = torchvision.datasets.MNIST(
    root='../data',
    train=True,
    transform=transform,
    download=True
)
train_ds

Dataset MNIST
    Number of datapoints: 60000
    Root location: ../data
    Split: Train

In [6]:
test_ds = torchvision.datasets.MNIST(
    root='../data',
    train=False,
    transform=transform
)
test_ds

Dataset MNIST
    Number of datapoints: 10000
    Root location: ../data
    Split: Test

In [7]:
train_dl = torch.utils.data.DataLoader(train_ds,bs,True,num_workers=12)
test_dl = torch.utils.data.DataLoader(test_ds,bs,False,num_workers=12)

In [8]:
input_size = train_ds[0][0].numel()

In [9]:
train_ds[0][0].size()

torch.Size([1, 28, 28])

In [10]:
def conv(ni,nf):
    return nn.Sequential(
        nn.Conv2d(ni,nf,kernel_size=3,stride=2,padding=1),
        nn.BatchNorm2d(nf),
        nn.ReLU(True)
    )

In [11]:
def resblock(nf):
    return nn.Sequential(
        nn.Conv2d(nf,nf,kernel_size=3),
        nn.Conv2d(nf,nf,kernel_size=3)
    )

In [25]:
class ResNet(nn.Module):
    def __init__(self,num_class):
        super().__init__()
        self.cnn1 = conv(1,16) #14
        self.res1 = res_block(16)
        self.cnn2 = conv(16,32)#7     
        self.res2 = res_block(32)
        self.cnn3 = conv(32,16)#4        
        self.res3 = res_block(16)
#         self.cnn4 = conv(32,16)#2       
#         self.res4 = res_block(16)
        self.cnn5 = conv(16,num_class)#1
    def forward(self,x):
        x = self.cnn1(x)
        x = x + self.res1(x)
        x = self.cnn2(x)        
        x = x + self.res2(x)
        x = self.cnn3(x)        
        x = x + self.res3(x)
#         x = self.cnn4(x)        
#         x = x + self.res4(x)
        x = self.cnn5(x)
        return x.view(x.size(0),-1)

In [26]:
model = ResNet(10)

In [27]:
images = next(iter(train_dl))

In [28]:
model(images[0]);

In [29]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=3e-3)

In [30]:
from tqdm import tqdm_notebook as tqdm

In [31]:
def train(model,dl,optimizer):
    model.cuda()
    model.train()
    t = tqdm(dl)
    for i,(images,labels) in enumerate(t):
        images,labels = images.cuda(),labels.cuda()
        optimizer.zero_grad()
        preds = model(images)
        loss = criterion(preds,labels)
        t.set_description(f'{loss.item()}')
        t.refresh()
        loss.backward()
        optimizer.step()
def test(model,dl):
    model.cuda()
    model.eval()
    correct = 0
    total = 0
    for img,y in dl:
        img,y = img.cuda(),y.cuda()
        preds = model(img)
        pred = preds.max(1, keepdim=True)[1]
        total += preds.size(0)
        correct += pred.eq(y.view_as(pred)).sum().item()
    return 1-(correct / len(dl.dataset))

In [ ]:
for i in range(5):
    train(model,train_dl,optimizer)
    test(model,test_dl)

In [32]:
def get_optimizer(trial, model):
    optimizer_names = ['Adam', 'MomentumSGD']
    optimizer_name = trial.suggest_categorical('optimizer', optimizer_names)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-10, 1e-3)
    if optimizer_name == optimizer_names[0]: 
        adam_lr = trial.suggest_loguniform('adam_lr', 1e-5, 1e-1)
        optimizer = optim.Adam(model.parameters(), lr=adam_lr, weight_decay=weight_decay)
    else:
        momentum_sgd_lr = trial.suggest_loguniform('momentum_sgd_lr', 1e-5, 1e-1)
        optimizer = optim.SGD(model.parameters(), lr=momentum_sgd_lr,
                              momentum=0.9, weight_decay=weight_decay)
    return optimizer

In [33]:
def objective_wrapper(pbar):
    def objective(trial):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        print(device)
        model = ResNet(10).cuda()
        optimizer = get_optimizer(trial, model)

        for step in range(EPOCH):
            train(model,train_dl, optimizer)
            error_rate = test(model, test_dl)

            trial.report(error_rate, step)
            if trial.should_prune(step):
                pbar.update()
                raise optuna.structs.TrialPruned()

        pbar.update()

        return error_rate
    
    return objective

In [34]:
TRIAL_SIZE = 50
EPOCH = 1

with tqdm(total=TRIAL_SIZE) as pbar:
    study = optuna.create_study(pruner=optuna.pruners.MedianPruner())
    study.optimize(objective_wrapper(pbar), n_trials=TRIAL_SIZE)

cuda


cuda


cuda


cuda


cuda


...

cuda


cuda


cuda


cuda


cuda


In [48]:
study.best_params

{'optimizer': 'Adam',
 'weight_decay': 0.00019131849419605393,
 'adam_lr': 0.0014739748148838122}

In [49]:
study.best_params['adam_lr']

0.0014739748148838122

In [50]:
study.best_value

0.01200000000000001

In [51]:
for i in range(5):
    train(model,train_dl,optim.Adam(model.parameters(),lr=study.best_params['adam_lr'],weight_decay=study.best_params['weight_decay']))

In [47]:
1-test(model,test_dl)

0.9901

In [53]:
df = study.trials_dataframe()

In [55]:
df.head()

number                state   value             datetime_start  \
                                                                   
0      0  TrialState.COMPLETE  0.1355 2019-07-15 23:51:38.487460   
1      1  TrialState.COMPLETE  0.0143 2019-07-15 23:52:09.109937   
2      2  TrialState.COMPLETE  0.0265 2019-07-15 23:52:39.824519   
3      3  TrialState.COMPLETE  0.1339 2019-07-15 23:53:04.154335   
4      4  TrialState.COMPLETE  0.0196 2019-07-15 23:53:28.655204   

           datetime_complete    params                               \
                               adam_lr momentum_sgd_lr    optimizer   
0 2019-07-15 23:52:09.109515  0.000090             NaN         Adam   
1 2019-07-15 23:52:39.823919  0.007205             NaN         Adam   
2 2019-07-15 23:53:04.153158       NaN        0.000971  MomentumSGD   
3 2019-07-15 23:53:28.654131       NaN        0.000100  MomentumSGD   
4 2019-07-15 23:53:52.791244       NaN        0.036752  MomentumSGD   

                system_attrs intermediate_values  
   weight_decay      _number                   0  
0  6.861286e-10            0              0.1355  
1  4.622720e-08            1              0.0143  
2  5.418758e-04            2              0.0265  
3  8.117865e-09            3              0.1339  
4  2.819793e-08            4              0.0196

In [57]:
df.sort_values(['value'])

number                state   value             datetime_start  \
                                                                    
24     24  TrialState.COMPLETE  0.0120 2019-07-16 00:03:14.357314   
1       1  TrialState.COMPLETE  0.0143 2019-07-15 23:52:09.109937   
16     16  TrialState.COMPLETE  0.0146 2019-07-15 23:59:11.601895   
9       9  TrialState.COMPLETE  0.0147 2019-07-15 23:55:42.031665   
27     27  TrialState.COMPLETE  0.0150 2019-07-16 00:04:46.124208   
18     18  TrialState.COMPLETE  0.0154 2019-07-16 00:00:12.170658   
12     12  TrialState.COMPLETE  0.0161 2019-07-15 23:57:07.994161   
40     40  TrialState.COMPLETE  0.0161 2019-07-16 00:10:55.952270   
30     30  TrialState.COMPLETE  0.0168 2019-07-16 00:06:13.166648   
25     25  TrialState.COMPLETE  0.0171 2019-07-16 00:03:44.951208   
38     38  TrialState.COMPLETE  0.0172 2019-07-16 00:09:53.757143   
37     37  TrialState.COMPLETE  0.0179 2019-07-16 00:09:29.009616   
23     23  TrialState.COMPLETE  0.0181 2019-07-16 00:02:43.375726   
10     10  TrialState.COMPLETE  0.0185 2019-07-15 23:56:06.290408   
32     32  TrialState.COMPLETE  0.0186 2019-07-16 00:07:08.475453   
22     22  TrialState.COMPLETE  0.0191 2019-07-16 00:02:12.935531   
4       4  TrialState.COMPLETE  0.0196 2019-07-15 23:53:28.655204   
34     34  TrialState.COMPLETE  0.0199 2019-07-16 00:08:03.126140   
48     48  TrialState.COMPLETE  0.0199 2019-07-16 00:14:49.046289   
49     49  TrialState.COMPLETE  0.0199 2019-07-16 00:15:13.321134   
20     20  TrialState.COMPLETE  0.0200 2019-07-16 00:01:12.812653   
19     19  TrialState.COMPLETE  0.0219 2019-07-16 00:00:42.814524   
8       8  TrialState.COMPLETE  0.0219 2019-07-15 23:55:11.641338   
15     15  TrialState.COMPLETE  0.0220 2019-07-15 23:58:40.480924   
14     14  TrialState.COMPLETE  0.0223 2019-07-15 23:58:09.621352   
11     11  TrialState.COMPLETE  0.0223 2019-07-15 23:56:36.841252   
5       5  TrialState.COMPLETE  0.0223 2019-07-15 23:53:52.792508   
46     46  TrialState.COMPLETE  0.0229 2019-07-16 00:13:47.688080   
47     47  TrialState.COMPLETE  0.0233 2019-07-16 00:14:18.105690   
45     45  TrialState.COMPLETE  0.0236 2019-07-16 00:13:17.243872   
42     42  TrialState.COMPLETE  0.0238 2019-07-16 00:11:51.332986   
6       6  TrialState.COMPLETE  0.0243 2019-07-15 23:54:23.571784   
7       7  TrialState.COMPLETE  0.0253 2019-07-15 23:54:47.633870   
2       2  TrialState.COMPLETE  0.0265 2019-07-15 23:52:39.824519   
44     44  TrialState.COMPLETE  0.0273 2019-07-16 00:12:46.225176   
41     41  TrialState.COMPLETE  0.0291 2019-07-16 00:11:20.450598   
26     26  TrialState.COMPLETE  0.0349 2019-07-16 00:04:15.490582   
36     36  TrialState.COMPLETE  0.0463 2019-07-16 00:08:58.098181   
29     29  TrialState.COMPLETE  0.0517 2019-07-16 00:05:48.036388   
35     35  TrialState.COMPLETE  0.0570 2019-07-16 00:08:33.526555   
13     13  TrialState.COMPLETE  0.0708 2019-07-15 23:57:38.845809   
21     21  TrialState.COMPLETE  0.1178 2019-07-16 00:01:42.798731   
33     33  TrialState.COMPLETE  0.1213 2019-07-16 00:07:39.102162   
3       3  TrialState.COMPLETE  0.1339 2019-07-15 23:53:04.154335   
0       0  TrialState.COMPLETE  0.1355 2019-07-15 23:51:38.487460   
17     17  TrialState.COMPLETE  0.1463 2019-07-15 23:59:41.872564   
31     31  TrialState.COMPLETE  0.3122 2019-07-16 00:06:44.402379   
39     39  TrialState.COMPLETE  0.4111 2019-07-16 00:10:24.838980   
28     28  TrialState.COMPLETE  0.6725 2019-07-16 00:05:16.706075   
43     43  TrialState.COMPLETE  0.8681 2019-07-16 00:12:22.231479   

            datetime_complete    params                               \
                                adam_lr momentum_sgd_lr    optimizer   
24 2019-07-16 00:03:44.942917  0.001474             NaN         Adam   
1  2019-07-15 23:52:39.823919  0.007205             NaN         Adam   
16 2019-07-15 23:59:41.868783  0.003975             NaN         Adam   
9  2019-07-15 23:56:06.288145       NaN        0.02131